In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import joblib
from sklearn.preprocessing import MinMaxScaler
import time
import warnings
warnings.simplefilter('ignore')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/脫硫劑優化改善/20240916-CFB2脫硫劑優化改善.xlsx',index_col=0,parse_dates=True)
print(df.shape)
df.sample(5)

(74593, 60)


,MLUT4_AIA792101A,MLUT4_AIA792101B,MLUT4_AIC-232B,MLUT4_AT-232A,MLUT4_AT-237,MLUT4_AT-240,MLUT4_AT-240A,MLUT4_FIC-231A,MLUT4_FIC-231B,MLUT4_FIC-231C,...,MLUT4_TE-252F,MLUT4_TE-252G,MLUT4_TE-252H,MLUT4_TE-252I,MLUT4_ZT-231,MLUT4_ZT-232,DeSOx_1st,DeSOx_2nd,前爐SOx濃度,鈣硫比
2024-03-24 14:35:00,20.225584,20.32125,0,5.967573,22.27136,0,0,0,0,0,...,31.9,25,25,25,100,100,NaN,NaN,NaN,NaN
2024-08-23 18:15:00,6.728925,5.6,32.626725,263.333092,9.733874,12.824073,12.830866,16.956642,0,14.770399,...,941.818432,909.790713,911.702348,671.958941,99.733861,16.142735,0.977385,0.606946,1442.673461,12.523086
2024-07-05 08:55:00,4.8,4.26385,69.118471,180.860771,7.727157,13.2,13.2,15.674875,13.385411,14.490561,...,909.070652,877.068876,912.062691,866.595614,63.335058,29,0.965656,0.809024,2012.510823,5.547344
2024-02-03 22:40:00,20.327526,20.388707,0,0,22.659882,0.1,0.1,0,0,0,...,26.862098,19.911149,19.9,19.9,28.712536,30.1,NaN,-inf,NaN,NaN
2024-04-08 17:10:00,6.72744,6.18823,88.501915,211.967802,9.861315,11.869701,11.869701,14.706634,9.414442,14.612828,...,906.863616,877.036716,925.671379,889.19199,73.67627,14.362702,0.958189,0.865882,2116.717187,3.257682


In [ ]:
df

,MLUT4_AIA792101A,MLUT4_AIA792101B,MLUT4_AIC-232B,MLUT4_AT-232A,MLUT4_AT-237,MLUT4_AT-240,MLUT4_AT-240A,MLUT4_FIC-231A,MLUT4_FIC-231B,MLUT4_FIC-231C,...,MLUT4_TE-252F,MLUT4_TE-252G,MLUT4_TE-252H,MLUT4_TE-252I,MLUT4_ZT-231,MLUT4_ZT-232,DeSOx_1st,DeSOx_2nd,前爐SOx濃度,鈣硫比
datetime,ECO O2_A #2,ECO O2_B #2,ECO SOx #2,ECO NOx #2,煙囪O2_#2,煙囪SOx #2,煙囪SOx-A #2,飼料機A#2,飼料機B#2,飼料機C#2,...,上層爐溫F#2,上層爐溫G#2,上層爐溫H#2,上層爐溫I#2,PAF風門開度#2,SAF風門開度#2,NaN,NaN,NaN,NaN
2024-01-01 00:00:00,5.540932,6.8,67.034424,151.208024,9.623605,8.619934,8.619934,20.605973,0,17.456094,...,876.984866,845.297084,845.703201,876.997523,59.200001,15,0.969941,0.871410,2230.093232,2.223152
2024-01-01 00:05:00,5.533478,6.8,66.662524,150.811453,9.616252,8.147002,8.147002,20.18102,0,17.449118,...,875.326466,846.587071,844.10285,877.38106,59.200001,15,0.969979,0.877787,2220.527929,2.220789
2024-01-01 00:10:00,5.526025,6.8,64.320729,151.708113,9.608899,8.247992,8.247992,20.200619,0,17.442142,...,875.675699,845.374599,843.500123,877.8976,59.200001,15,0.971249,0.871768,2237.129686,2.217612
2024-01-01 00:15:00,5.518571,6.801543,64.840762,151.283528,9.600898,7.60582,7.60582,20.220218,0,17.571883,...,875.435828,845.10045,842.910508,874.888616,59.200001,15,0.970896,0.882700,2227.895501,2.214444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-15 23:35:00,6.258452,5.140339,114.166438,209.520746,9.360355,13.105704,13.105704,12.795748,11.854892,12.8422,...,916.635699,889.885541,906.016252,870.865216,100,26.103367,0.945172,0.885205,2082.274527,3.928554
2024-09-15 23:40:00,6.284992,5.166336,106.669841,208.177318,9.280099,9.903673,9.903673,12.797045,11.862273,12.823741,...,914.06561,889.042164,904.255862,869.007829,100,26.093265,0.948517,0.907156,2071.948976,3.927730
2024-09-15 23:45:00,6.232171,5.225976,104.663595,211.564232,9.359701,9.345572,9.345572,12.798342,11.880556,12.842696,...,913.978871,890.439554,903.442876,867.71297,100,26.1,0.949882,0.910708,2088.339754,3.928170
2024-09-15 23:50:00,6.20333,5.14117,103.246788,206.570014,9.211132,8.303164,8.303164,12.799638,11.926489,12.825216,...,913.419379,887.270132,903.808073,867.684024,100,26.078118,0.949256,0.919579,2034.665472,3.927928


In [ ]:
df = df.iloc[1:,:]
df

,MLUT4_AIA792101A,MLUT4_AIA792101B,MLUT4_AIC-232B,MLUT4_AT-232A,MLUT4_AT-237,MLUT4_AT-240,MLUT4_AT-240A,MLUT4_FIC-231A,MLUT4_FIC-231B,MLUT4_FIC-231C,...,MLUT4_TE-252F,MLUT4_TE-252G,MLUT4_TE-252H,MLUT4_TE-252I,MLUT4_ZT-231,MLUT4_ZT-232,DeSOx_1st,DeSOx_2nd,前爐SOx濃度,鈣硫比
2024-01-01 00:00:00,5.540932,6.8,67.034424,151.208024,9.623605,8.619934,8.619934,20.605973,0,17.456094,...,876.984866,845.297084,845.703201,876.997523,59.200001,15,0.969941,0.871410,2230.093232,2.223152
2024-01-01 00:05:00,5.533478,6.8,66.662524,150.811453,9.616252,8.147002,8.147002,20.18102,0,17.449118,...,875.326466,846.587071,844.10285,877.38106,59.200001,15,0.969979,0.877787,2220.527929,2.220789
2024-01-01 00:10:00,5.526025,6.8,64.320729,151.708113,9.608899,8.247992,8.247992,20.200619,0,17.442142,...,875.675699,845.374599,843.500123,877.8976,59.200001,15,0.971249,0.871768,2237.129686,2.217612
2024-01-01 00:15:00,5.518571,6.801543,64.840762,151.283528,9.600898,7.60582,7.60582,20.220218,0,17.571883,...,875.435828,845.10045,842.910508,874.888616,59.200001,15,0.970896,0.882700,2227.895501,2.214444
2024-01-01 00:20:00,5.511117,6.819977,64.390831,150.994896,9.581994,6.110485,6.110485,20.549012,0,17.339679,...,875.2454,846.058978,844.200021,878.41671,59.200001,15,0.970878,0.905103,2211.052236,2.211285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-15 23:35:00,6.258452,5.140339,114.166438,209.520746,9.360355,13.105704,13.105704,12.795748,11.854892,12.8422,...,916.635699,889.885541,906.016252,870.865216,100,26.103367,0.945172,0.885205,2082.274527,3.928554
2024-09-15 23:40:00,6.284992,5.166336,106.669841,208.177318,9.280099,9.903673,9.903673,12.797045,11.862273,12.823741,...,914.06561,889.042164,904.255862,869.007829,100,26.093265,0.948517,0.907156,2071.948976,3.927730
2024-09-15 23:45:00,6.232171,5.225976,104.663595,211.564232,9.359701,9.345572,9.345572,12.798342,11.880556,12.842696,...,913.978871,890.439554,903.442876,867.71297,100,26.1,0.949882,0.910708,2088.339754,3.928170
2024-09-15 23:50:00,6.20333,5.14117,103.246788,206.570014,9.211132,8.303164,8.303164,12.799638,11.926489,12.825216,...,913.419379,887.270132,903.808073,867.684024,100,26.078118,0.949256,0.919579,2034.665472,3.927928


In [ ]:
import pandas as pd

def missing_value_summary(df):
    missing_count = df.isnull().sum()
    missing_percent = df.isnull().mean() * 100  # 乘 100 轉換為百分比

    summary_df = pd.DataFrame({
        '欄位名稱': missing_count.index,
        '缺失數': missing_count.values,
        '缺失百分比': missing_percent.values
    })

    # 只顯示有缺失值的欄位，並依照缺失百分比由高到低排序
    summary_df = summary_df[summary_df['缺失數'] > 0]
    summary_df = summary_df.sort_values(by='缺失百分比', ascending=False).reset_index(drop=True)

    return summary_df
missing_value_summary(df)

,欄位名稱,缺失數,缺失百分比
0,MLUT4_SIV-B7921,67848,90.958816
1,DeSOx_1st,18854,25.276169
2,鈣硫比,18798,25.201094
3,前爐SOx濃度,18054,24.203668
4,DeSOx_2nd,10351,13.876823
5,MLUT4_AIA792101B,2,0.002681
6,MLUT4_AT-232A,2,0.002681
7,MLUT4_AT-237,2,0.002681
8,MLUT4_AIA792101A,2,0.002681
9,MLUT4_FIC-233,2,0.002681


# 刪除90%缺失值的欄位

In [ ]:
df = df.drop(columns=['MLUT4_SIV-B7921'])
missing_value_summary(df)

,欄位名稱,缺失數,缺失百分比
0,DeSOx_1st,18854,25.276169
1,鈣硫比,18798,25.201094
2,前爐SOx濃度,18054,24.203668
3,DeSOx_2nd,10351,13.876823
4,MLUT4_AIA792101A,2,0.002681
5,MLUT4_AIA792101B,2,0.002681
6,MLUT4_AT-232A,2,0.002681
7,MLUT4_AT-237,2,0.002681
8,MLUT4_FIC-231B,2,0.002681
9,MLUT4_FIC-231A,2,0.002681


# replace無限大

In [ ]:
import numpy as np

df = df.replace([np.inf, -np.inf], np.nan)  # 將無限大轉為 NaN

# 特徵縮放

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mm_x = MinMaxScaler()
mm_y = MinMaxScaler()

y_col = 'MLUT4_AT-240'
x_col = df.drop(columns=[y_col]).columns

df[x_col] = mm_x.fit_transform(df[x_col])
df[y_col] = mm_y.fit_transform(df[[y_col]])

df

,MLUT4_AIA792101A,MLUT4_AIA792101B,MLUT4_AIC-232B,MLUT4_AT-232A,MLUT4_AT-237,MLUT4_AT-240,MLUT4_AT-240A,MLUT4_FIC-231A,MLUT4_FIC-231B,MLUT4_FIC-231C,...,MLUT4_TE-252F,MLUT4_TE-252G,MLUT4_TE-252H,MLUT4_TE-252I,MLUT4_ZT-231,MLUT4_ZT-232,DeSOx_1st,DeSOx_2nd,前爐SOx濃度,鈣硫比
2024-01-01 00:00:00,0.263946,0.322219,0.074442,0.507117,0.378828,0.067343,0.067343,0.851586,0.000000,0.741053,...,0.682882,0.878104,0.876123,0.876262,0.589078,0.137056,0.999984,0.999713,0.013288,0.000013
2024-01-01 00:05:00,0.263591,0.322219,0.074029,0.505787,0.378531,0.063648,0.063648,0.834024,0.000000,0.740757,...,0.681560,0.879465,0.874439,0.876651,0.589078,0.137056,0.999984,0.999727,0.013231,0.000013
2024-01-01 00:10:00,0.263236,0.322219,0.071428,0.508794,0.378234,0.064437,0.064437,0.834834,0.000000,0.740461,...,0.681839,0.878186,0.873805,0.877174,0.589078,0.137056,0.999985,0.999714,0.013330,0.000013
2024-01-01 00:15:00,0.262881,0.322292,0.072006,0.507370,0.377911,0.059420,0.059420,0.835644,0.000000,0.745969,...,0.681648,0.877896,0.873184,0.874126,0.589078,0.137056,0.999985,0.999738,0.013275,0.000013
2024-01-01 00:20:00,0.262526,0.323171,0.071506,0.506402,0.377147,0.047738,0.047738,0.849232,0.000000,0.736111,...,0.681496,0.878908,0.874541,0.877700,0.589078,0.137056,0.999984,0.999788,0.013174,0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-15 23:35:00,0.298126,0.243111,0.126782,0.702684,0.368193,0.102388,0.102388,0.528812,0.480902,0.545182,...,0.714474,0.925158,0.939581,0.870050,1.000000,0.249780,0.999971,0.999744,0.012407,0.000023
2024-09-15 23:40:00,0.299390,0.244350,0.118457,0.698179,0.364951,0.077372,0.077372,0.528865,0.481202,0.544399,...,0.712426,0.924268,0.937728,0.868168,1.000000,0.249678,0.999973,0.999793,0.012345,0.000023
2024-09-15 23:45:00,0.296874,0.247193,0.116229,0.709538,0.368167,0.073012,0.073012,0.528919,0.481943,0.545203,...,0.712357,0.925742,0.936873,0.866857,1.000000,0.249746,0.999973,0.999801,0.012443,0.000023
2024-09-15 23:50:00,0.295500,0.243151,0.114656,0.692788,0.362165,0.064868,0.064868,0.528972,0.483807,0.544461,...,0.711911,0.922398,0.937257,0.866827,1.000000,0.249524,0.999973,0.999820,0.012123,0.000023


# 建立資料集 for 建模使用

In [ ]:
def build_dataset(df,n):
    data = {'X':[],'y':[],'idx':[]}
    for i in tqdm(range(0,len(df)-n)):
        curr = df.iloc[i:i+n] #一小段資料
        if np.isnan(curr.values).sum() == 0: #確保這段資料無缺失值
            data['X'].append(curr[x_col].values)# X
            data['y'].append(curr[y_col].values[-1]) #y
            data['idx'].append(curr.index[-1]) #這段時間最後 出料對應的時間點
    # reshape
    data['X'] = np.array(data['X'])
    data['y'] = np.array(data['y']).reshape(-1,1)
    # store mm_scaler
    data['mm_x'] = mm_x
    data['mm_y'] = mm_y
    # store col_name
    data['x_col'] = x_col
    data['y_col'] = y_col
    return data

In [ ]:
data = build_dataset(df,10)

  0%|          | 0/74582 [00:00<?, ?it/s]

In [ ]:
print(len(data['idx']))

54187


In [ ]:
joblib.dump(data,'dataset.pkl')

['dataset.pkl']